In [1]:
import random
import pyspark.sql.functions as F

In [2]:
nRegisters = 100000
header = ['cod_estado', 'sexo', 'estado_civil']
var = [[x,y,z] for x,y,z in  zip([random.choice(range(11,54)) for _ in range(nRegisters)], [random.choice(['F', 'M']) for _ in range(nRegisters)], [random.choice(['SOLTEIRO', 'CASADO', 'DIVORCIADO']) for _ in range(nRegisters)])]
groupby_var = 'cod_estado'
base_name = 'base1'
df = spark.createDataFrame(var, header)

In [3]:
df.show()

+----------+----+------------+
|cod_estado|sexo|estado_civil|
+----------+----+------------+
|        27|   M|    SOLTEIRO|
|        13|   F|  DIVORCIADO|
|        11|   F|    SOLTEIRO|
|        33|   F|  DIVORCIADO|
|        14|   M|      CASADO|
|        47|   F|  DIVORCIADO|
|        42|   F|    SOLTEIRO|
|        33|   M|      CASADO|
|        39|   M|  DIVORCIADO|
|        45|   M|  DIVORCIADO|
|        16|   F|      CASADO|
|        49|   F|    SOLTEIRO|
|        45|   F|    SOLTEIRO|
|        40|   M|      CASADO|
|        30|   F|      CASADO|
|        35|   F|    SOLTEIRO|
|        33|   M|    SOLTEIRO|
|        37|   M|    SOLTEIRO|
|        53|   F|  DIVORCIADO|
|        20|   M|      CASADO|
+----------+----+------------+
only showing top 20 rows



In [4]:
agg = df.groupby(header).count()
agg_concat = agg.withColumn('agg_concat', F.concat_ws('_', *[x for x in header if x != groupby_var]))
agg_concat = agg_concat.withColumn('agg_concat', F.concat_ws('_', F.lit(base_name), 'agg_concat'))

In [5]:
agg_concat.show()

+----------+----+------------+-----+------------------+
|cod_estado|sexo|estado_civil|count|        agg_concat|
+----------+----+------------+-----+------------------+
|        26|   F|      CASADO|  366|    base1_F_CASADO|
|        40|   F|  DIVORCIADO|  387|base1_F_DIVORCIADO|
|        22|   F|      CASADO|  380|    base1_F_CASADO|
|        16|   F|  DIVORCIADO|  383|base1_F_DIVORCIADO|
|        12|   F|      CASADO|  411|    base1_F_CASADO|
|        37|   F|    SOLTEIRO|  427|  base1_F_SOLTEIRO|
|        51|   F|      CASADO|  376|    base1_F_CASADO|
|        48|   M|      CASADO|  364|    base1_M_CASADO|
|        18|   F|    SOLTEIRO|  383|  base1_F_SOLTEIRO|
|        14|   M|  DIVORCIADO|  360|base1_M_DIVORCIADO|
|        41|   M|  DIVORCIADO|  360|base1_M_DIVORCIADO|
|        37|   F|      CASADO|  391|    base1_F_CASADO|
|        25|   M|  DIVORCIADO|  403|base1_M_DIVORCIADO|
|        46|   F|  DIVORCIADO|  373|base1_F_DIVORCIADO|
|        22|   M|  DIVORCIADO|  389|base1_M_DIVO

In [6]:
result = agg_concat.groupby(groupby_var).pivot('agg_concat').sum('count').orderBy(groupby_var)

In [7]:
result.show(1000, False)

+----------+--------------+------------------+----------------+--------------+------------------+----------------+
|cod_estado|base1_F_CASADO|base1_F_DIVORCIADO|base1_F_SOLTEIRO|base1_M_CASADO|base1_M_DIVORCIADO|base1_M_SOLTEIRO|
+----------+--------------+------------------+----------------+--------------+------------------+----------------+
|11        |375           |375               |356             |384           |369               |417             |
|12        |411           |368               |384             |390           |369               |385             |
|13        |396           |359               |380             |397           |396               |378             |
|14        |369           |390               |391             |426           |360               |384             |
|15        |370           |373               |411             |385           |332               |371             |
|16        |362           |383               |336             |408           |37